In [ ]:
### this example demonstrates how to use kfp python sdk to create pipeline / update its versions and trigger runs
### to see full doc on how we can use python sdk, please refer to: 
### https://kubeflow-pipelines.readthedocs.io/en/stable/index.html

In [7]:
with open("requirements.txt", "w") as f:
    f.write("kfp==1.4.0\n")
    
!pip install -r requirements.txt  --upgrade --user

In [1]:
# as codes are executed on kernel of jupyter, a pod on kubernetes cluster
# so it should use internal dns for looking up global entry from istio-system
host = 'http://istio-ingressgateway.istio-system.svc.cluster.local:80/pipeline'
namespace = 'kubeflow-user-example-com'

### update your authservice_session everytime when you logged into kubeflow.
### trigger your developer console and to go application tab, you can see cookies section.
authservice_session = 'authservice_session=MTYxOTM1ODY5OHxOd3dBTkZKQk1sbzJTalZDVkZCUlVVNUhOazlITmxvMlZEWkJSVWROTkVkSlRrOVlVMEZOUWtFeVNGVkJSVk5aUmxNMVRsTlRSbEU9fC6ymz93WelmasdGsQ7yT83LxKNL3Y4kLvmMLCmQaPd1'



In [2]:
import kfp
import json
import logging

client = kfp.Client(host=host, cookies=authservice_session)
list_exps = client.list_experiments(namespace=namespace)
print(list_exps)

{'experiments': [{'created_at': datetime.datetime(2021, 4, 25, 14, 15, 26, tzinfo=tzlocal()),
                  'description': None,
                  'id': '8c0bbe27-e4e2-4d53-ab11-7dd3c3b11c4d',
                  'name': 'exp4echo',
                  'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'},
                 {'created_at': datetime.datetime(2021, 4, 25, 14, 20, 34, tzinfo=tzlocal()),
                  'description': None,
                  'id': '547c0102-ff8f-454d-ace9-648963640b79',
                  'name': 'exp4flipcoin',
                  'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                                   'type': 'NAMESPACE'},
                   

In [3]:
pipeline_template_path = "helloworld.zip"
pipeline_name = "helloworld"
pipeline_description = "some descriptions..."

In [4]:
# create an pipeline with name/description/template local path

upload_pipeline = client.pipeline_uploads.upload_pipeline(pipeline_template_path,
    name=pipeline_name,
    description=pipeline_description,
    async_req=False,
)
print(upload_pipeline)

{'created_at': datetime.datetime(2021, 4, 25, 15, 34, 52, tzinfo=tzlocal()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2021, 4, 25, 15, 34, 52, tzinfo=tzlocal()),
                     'id': '33e65ed3-9e56-4b0c-8c01-33cf23895e8f',
                     'name': 'helloworld',
                     'package_url': None,
                     'parameters': None,
                     'resource_references': [{'key': {'id': '33e65ed3-9e56-4b0c-8c01-33cf23895e8f',
                                                      'type': 'PIPELINE'},
                                              'name': None,
                                              'relationship': 'OWNER'}]},
 'description': 'some descriptions...',
 'error': None,
 'id': '33e65ed3-9e56-4b0c-8c01-33cf23895e8f',
 'name': 'helloworld',
 'parameters': None,
 'resource_references': None,
 'url': None}


In [5]:
# update pipeline version
pipeline_id=upload_pipeline.id

import datetime
timed_version = datetime.datetime.today().strftime("%d/%m/%Y %H:%M:%S")

upload_pipeline_version = client.pipeline_uploads.upload_pipeline_version(pipeline_template_path,
    name=timed_version,
    pipelineid=pipeline_id,
    async_req=False,
)
print(upload_pipeline_version)

{'code_source_url': None,
 'created_at': datetime.datetime(2021, 4, 25, 15, 34, 56, tzinfo=tzlocal()),
 'id': '886d27de-feb7-4986-8a6f-a1e52e573393',
 'name': '25/04/2021 15:34:56',
 'package_url': None,
 'parameters': None,
 'resource_references': [{'key': {'id': '33e65ed3-9e56-4b0c-8c01-33cf23895e8f',
                                  'type': 'PIPELINE'},
                          'name': None,
                          'relationship': 'OWNER'}]}


In [6]:
### trigger run on a specific pipeline
experimentid=list_exps.experiments[0].id
versionid=upload_pipeline_version.id
job_name = "run helloworld"

run_resp = client.run_pipeline(
    experimentid,
    job_name,
    pipeline_id=pipeline_id,
    version_id=versionid,
)
print(run_resp)

{'created_at': datetime.datetime(2021, 4, 25, 15, 35, 3, tzinfo=tzlocal()),
 'description': None,
 'error': None,
 'finished_at': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
 'id': '50404b38-e3a8-47f2-8c52-03ef27e35a54',
 'metrics': None,
 'name': 'run helloworld',
 'pipeline_spec': {'parameters': None,
                   'pipeline_id': '33e65ed3-9e56-4b0c-8c01-33cf23895e8f',
                   'pipeline_manifest': None,
                   'pipeline_name': 'helloworld',
                   'workflow_manifest': '{"kind":"Workflow","apiVersion":"argoproj.io/v1alpha1","metadata":{"generateName":"launch-katib-early-stopping-experiment-","creationTimestamp":null,"labels":{"pipelines.kubeflow.org/kfp_sdk_version":"1.4.0"},"annotations":{"pipelines.kubeflow.org/kfp_sdk_version":"1.4.0","pipelines.kubeflow.org/pipeline_compilation_time":"2021-04-25T15:32:59.214228","pipelines.kubeflow.org/pipeline_spec":"{\\"description\\": '
                                        '\\"An example to 